#### GMM_select peak

In [70]:
def calc_causalSNP(snv, si_list):
    
    import math
    
    # si = np.array([0.04,0.0900007,0.8064,0.0784,0.10240,0.04834,0.9804])
    si = np.array(si_list)
    X = np.reshape(si,(len(si),1))
    # N = np.arange(1, NNN+1)
    N = np.arange(1, 5) ### 不能超过样本的数目
    models = [None for i in range(len(N))]

    for i in range(len(N)):
        models[i] = mixture.GaussianMixture(N[i]).fit(X)
    # compute the BIC
    BIC = [m.bic(X) for m in models]
    # best-fit mixture
    M_best = models[np.argmin(BIC)] #### BIC最低的模型是最好的

    # the largest peak in the distri
    #  score: returns an array of Log probabilities of each data point in X
    #  np.exp(logProb) => gives p(x) -> y-axis of the density or histogram
    #  X: si numbers
    logp = M_best.score_samples(X)
    p = np.exp(logp)
    maxP = np.argmax(p) #### index of largest prob
    peakX = X[maxP][0]  #### maxP在X中对应的si值
    nComp = M_best.n_components
    labs =  M_best.predict(X)
    peakComp = labs[maxP] #### component of largest si
    ### peakX ~ peakComp

    #randomly distribute the data points (评估background分布)
    #discretize the x-axis to bins: https://stackoverflow.com/questions/31730028/how-can-i-generate-a-random-sample-of-bin-counts-given-a-sequence-of-bin-probabi
    # nbin = BIN
    nbin = 50
    targBINindex = int(peakX*nbin)-1
    bgprob = [0] + [1./nbin for i in range(nbin)] ### 均匀分布
    # cdf = np.cumsum(bgprob)

    thresh=[]
    trials=500
    for i in range(trials):
        ## distributeBins (smartFunctions script): return bg list and bg (mean(bg) + variance(bg));
        ## targBINindex bin中data point的数目
        thresh.append(int(math.ceil(distributeBins(nbin,len(si),bgprob,targBINindex)[1])))

    # het = res[(snv, exon, tag)]['0/1'] + res[(snv, exon, tag)]['1/0']
    # tot = res[(snv, exon, tag)]['0/0'] + het + res[(snv, exon, tag)]['1/1']

    # do for all components
    # To decide whether the peakX is close to Si = 0 or 1
    # calculate the z-score of 0 or 1 to the peak component's mean
    zs, ps, means, stdevs, pns, ratios = [], [], [], [], [], []
    for i in range(nComp): ## 三个成分
        peakM = X[labs == i] ## 每一个成分对应的Si
        peakMn = len(peakM)  ## 每个成分包含数据点的个数
        mean = np.mean(peakM)
        stdev = np.std(peakM, dtype=np.float64)
        z0 = (0 - mean)/stdev
        p0 = sp.stats.norm.sf(abs(z0))*2 #twosided
        z1 = (1 - mean)/stdev
        p1 = sp.stats.norm.sf(abs(z1))*2 #twosided
        means.append(str(round(mean,4)))
        stdevs.append(str(round(stdev,4)))
        pns.append(str(peakMn))

    #     ### rm.bg ###
        thispeak = defaultdict(list) #len(p[labs == i])
        try: ### ind作为thispeak这个dict的一个key(因为取整，所以会对应一系列相近的si数据点)
            ### 这里取的是离mean最近的一个bin(是不是可以近似认为是包含mean的这个bin)；
            ### 用这个bin代表整个si分布，去和background分布去比较；
            ind = int(X[labs == i][np.argmin(abs(peakM-mean))][0]*nbin)
        except ValueError:
            zs.append('NA;NA')
            ps.append('NA;NA')
            continue
        for xxx in X[labs == i]:
            thispeak[int(xxx[0]*nbin)].append(xxx[0])

        #real peak needs to be > than mean + 2.58 times the stdev (99% confidence intervals 
        # assuming 500 trials give us normal distri)
        testingp = len(thispeak[ind])
        if testingp <= np.mean(thresh) + 2.58*np.std(thresh, dtype=np.float64, axis=0):
            zs.append('NA;NA')
            ps.append('NA;NA')
            continue
        #############
        ## 
        zs.append('{};{}'.format(z0,z1))
        ps.append('{};{}'.format(p0,p1))
    # out.write('{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\n'.format(snv,res[(snv,exon,tag)]['info'],exon,tag,
    #         tot,res[(snv,exon,tag)]['0/0'],het,res[(snv,exon,tag)]['1/1'],peakX,
    #         '|'.join(zs),'|'.join(ps),nComp,'|'.join(means),'|'.join(stdevs),'|'.join(pns)))
    print(snv, peakX, '|'.join(zs),'|'.join(ps),nComp,'|'.join(means),'|'.join(stdevs),'|'.join(pns))